# WebRTC

In [ ]:
# !apt-get install ffmpeg

In [ ]:
import webrtcvad
import collections
import contextlib
import wave
import os
import subprocess
from pydub import AudioSegment

# Convert MP3 to WAV
def convert_mp3_to_wav(mp3_path, wav_path):
    audio = AudioSegment.from_mp3(mp3_path)
    audio = audio.set_frame_rate(16000)  # Resample to 16000 Hz
    audio = audio.set_channels(1)  # Ensure audio is mono
    audio.export(wav_path, format="wav")

# Frame class to hold audio data
class Frame(object):
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration

def read_wave(path):
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate

def frame_generator(frame_duration_ms, audio, sample_rate):
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n

def vad_collector(sample_rate, frame_duration_ms, padding_duration_ms, vad, frames):
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    triggered = False

    voiced_frames = []
    intervals = []

    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                intervals.append((ring_buffer[0][0].timestamp, None))
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                intervals[-1] = (intervals[-1][0], frame.timestamp + frame.duration)
                triggered = False
                ring_buffer.clear()

    if triggered:
        intervals[-1] = (intervals[-1][0], frame.timestamp + frame.duration)

    return intervals

def save_audio_with_intervals(wav_path, intervals, output_with_voice, output_with_silence):
    audio = AudioSegment.from_wav(wav_path)
    silent_audio = AudioSegment.silent(duration=len(audio))

    audio_with_voice = silent_audio
    audio_with_silence = audio

    for start, end in intervals:
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)
        audio_with_voice = audio_with_voice.overlay(audio[start_ms:end_ms], position=start_ms)
        audio_with_silence = audio_with_silence.overlay(silent_audio[start_ms:end_ms], position=start_ms)

    audio_with_voice.export(output_with_voice, format="wav")
    audio_with_silence.export(output_with_silence, format="wav")

# Paths
mp3_path = os.path.join("data","XC240120 - Soundscape.mp3")  # Upload your MP3 file to Colab
wav_path = "temp_audio.wav"
output_with_voice = "output_with_voice_WebRTC.wav"
output_with_silence = "output_with_silence_WebRTC.wav"

# Convert MP3 to WAV
convert_mp3_to_wav(mp3_path, wav_path)

# Read WAV file
audio, sample_rate = read_wave(wav_path)

# Initialize VAD
vad = webrtcvad.Vad(3)

# Generate frames
frames = frame_generator(30, audio, sample_rate)
frames = list(frames)

# Collect voiced segments
intervals = vad_collector(sample_rate, 30, 300, vad, frames)

# Print intervals
for start, end in intervals:
    print(f"Start: {start:.2f}s, End: {end:.2f}s")

# Save new audio files with intervals
save_audio_with_intervals(wav_path, intervals, output_with_voice, output_with_silence)

# Cleanup temporary WAV file
os.remove(wav_path)

# Hugging Face Transformers

In [ ]:

from pyannote.audio import Pipeline
from pydub import AudioSegment
import numpy as np
import os
from huggingface_hub import login
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import pipeline as hf_pipeline
from dotenv import load_dotenv

load_dotenv()

# Read your Hugging Face token from environment. Do NOT hardcode secrets.
HF_TOKEN = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
if not HF_TOKEN:
    raise RuntimeError(
        "Missing Hugging Face token. Set HF_TOKEN or HUGGINGFACE_HUB_TOKEN in your environment."
    )

# Optionally ensure the token is active for the session (no-op if already logged in)
try:
    login(HF_TOKEN, add_to_git_credential=False)
except Exception:
    pass

# Model identifier for VAD (pyannote 3.1-compatible)
# Use the segmentation model directly instead of the pipeline
from pyannote.audio import Model

model_name = "pyannote/segmentation-3.0"
model = Model.from_pretrained(model_name, token=HF_TOKEN)

# Create a simple VAD pipeline using the segmentation model
from pyannote.audio.pipelines import VoiceActivityDetection

pipeline = VoiceActivityDetection(segmentation=model)

# Instantiate the pipeline with default parameters
HYPER_PARAMETERS = {
    "min_duration_on": 0.1,
    "min_duration_off": 0.1,
}
pipeline.instantiate(HYPER_PARAMETERS)

# Function to load and preprocess audio
def load_and_preprocess_audio(audio_path):
    """Loads and preprocesses audio for VAD.  Resamples to 16kHz mono, converts to NumPy array."""
    try:
        audio = AudioSegment.from_file(audio_path)
    except Exception as e:
        print(f"Error loading audio file {audio_path}: {e}")
        return None, None, None

    audio = audio.set_channels(1)  # Convert to mono
    audio = audio.set_frame_rate(16000)  # Resample to 16kHz
    samples = np.array(audio.get_array_of_samples())
    samples = samples.astype(np.float32) / np.iinfo(np.int16).max  # Normalize
    return samples, audio, audio.frame_rate

# Function to perform VAD
def detect_speech_intervals(samples, sample_rate, pipeline):
    """Performs voice activity detection on the audio."""

    try:
        # The pipeline expects a dictionary with 'waveform' and 'sample_rate'
        input_data = {"waveform": torch.from_numpy(samples).unsqueeze(0), "sample_rate": sample_rate}
        vad_result = pipeline(input_data)
    except Exception as e:
        print(f"Error during VAD: {e}")
        return [], 0.0

    intervals = []
    for segment in vad_result.get_timeline():
        intervals.append((segment.start, segment.end))
    return intervals, 0.0

# Function to save audio with intervals
def save_audio_with_intervals(audio, intervals, output_with_voice, output_with_silence, duration):
    silent_audio = AudioSegment.silent(duration=len(audio))

    audio_with_voice = silent_audio
    audio_with_silence = audio

    for start, end in intervals:
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)
        audio_with_voice = audio_with_voice.overlay(audio[start_ms:end_ms], position=start_ms)
        audio_with_silence = audio_with_silence.overlay(silent_audio[start_ms:end_ms], position=start_ms)

    audio_with_voice.export(output_with_voice, format="wav")
    audio_with_silence.export(output_with_silence, format="wav")

# Paths
# Update this to a valid local path on your machine
mp3_path = os.path.join("data","XC240120 - Soundscape.mp3") 

# Validate input path early
if not os.path.isfile(mp3_path):
    raise FileNotFoundError(
        f"Audio file not found: {mp3_path}. Update 'mp3_path' to a valid local file."
    )
output_with_voice = "output_with_voice_huggingface.wav"
output_with_silence = "output_with_silence_huggingface.wav"

# Load and preprocess audio
samples, audio_segment, sample_rate = load_and_preprocess_audio(mp3_path)

# Detect speech intervals
speech_intervals, duration = detect_speech_intervals(samples, sample_rate, pipeline)
duration = len(audio_segment) / 1000.0

print(speech_intervals)

# Save new audio files with intervals
save_audio_with_intervals(audio_segment, speech_intervals, output_with_voice, output_with_silence, duration)


## Old version performance evaluation

In [ ]:
from pyannote.audio import Pipeline
from pydub import AudioSegment
import numpy as np
import os  # For file existence check

In [ ]:
# Load the pre-trained VAD pipeline
try:
    pipeline = Pipeline.from_pretrained("pyannote/voice-activity-detection", use_auth_token=True)
except Exception as e:
    print(f"Error loading pipeline: {e}")
    exit()

# Function to load audio
def load_audio(audio_path):
    audio = AudioSegment.from_file(audio_path)
    return audio

# Function to perform VAD
def detect_speech_intervals(audio_path, pipeline):
    audio = load_audio(audio_path)
    duration = len(audio) / 1000.0  # Convert to seconds
    vad_result = pipeline({"uri": audio_path, "audio": audio_path})

    intervals = []
    for segment in vad_result.get_timeline():
        intervals.append((segment.start, segment.end))
    return intervals, duration

# Function to load manual labels from file
def load_manual_labels(label_file):
    """Loads manual labels from the given text file."""
    labels = []
    try:
        with open(label_file, 'r') as f:
            for line in f:
                start, end, label = line.strip().split('\t')
                labels.append((float(start), float(end)))
    except FileNotFoundError:
        print(f"Error: Label file not found: {label_file}")
        return None
    except ValueError:
        print(f"Error: Invalid format in label file {label_file}.  Expecting tab-separated start, end, label.")
        return None
    return labels

# Function to calculate precision and recall
def calculate_precision_recall_f1(vad_intervals, manual_intervals):
    """Calculates precision, recall, and F1-score."""

    # Convert intervals to sets of time points for easier comparison
    def intervals_to_set(intervals):
        time_points = set()
        for start, end in intervals:
            for i in range(int(start * 100), int(end * 100)):  # Convert to 10ms resolution
                time_points.add(i)
        return time_points

    vad_set = intervals_to_set(vad_intervals)
    manual_set = intervals_to_set(manual_intervals)

    true_positives = len(vad_set.intersection(manual_set))
    predicted_positives = len(vad_set)
    actual_positives = len(manual_set)

    precision = true_positives / predicted_positives if predicted_positives > 0 else 0
    recall = true_positives / actual_positives if actual_positives > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return precision, recall, f1

# Function to save VAD intervals to a file, including performance metrics
def save_vad_intervals(vad_intervals, manual_intervals, output_file):
    """Saves VAD intervals to a text file, including performance metrics."""
    try:
        with open(output_file, 'w') as f:
            # Calculate performance metrics
            precision, recall, f1 = calculate_precision_recall_f1(vad_intervals, manual_intervals)

            # Write header with performance metrics
            f.write(f"# Precision: {precision:.4f}\n")
            f.write(f"# Recall: {recall:.4f}\n")
            f.write(f"# F1-score: {f1:.4f}\n")
            f.write("# start\tend\tVAD\n") # Header for intervals

            # Write VAD intervals
            for start, end in vad_intervals:
                f.write(f"{start}\t{end}\tVAD\n")

        print(f"VAD intervals and performance metrics saved to {output_file}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-score: {f1:.4f}")


    except Exception as e:
        print(f"Error saving VAD intervals: {e}")

# Function to save audio with intervals
def save_audio_with_intervals(audio, intervals, output_with_voice, output_with_silence, duration):
    silent_audio = AudioSegment.silent(duration=len(audio))

    audio_with_voice = silent_audio
    audio_with_silence = audio

    for start, end in intervals:
        start_ms = int(start * 1000)
        end_ms = int(end * 1000)
        audio_with_voice = audio_with_voice.overlay(audio[start_ms:end_ms], position=start_ms)
        audio_with_silence = audio_with_silence.overlay(silent_audio[start_ms:end_ms], position=start_ms)

    audio_with_voice.export(output_with_voice, format="wav")
    audio_with_silence.export(output_with_silence, format="wav")

# Paths
mp3_path = "drive/MyDrive/datasets/audio_ambiental_narracion/XC237810_1 - Soundscape.mp3"  # Replace with your path
manual_label_file = "drive/MyDrive/datasets/audio_ambiental_narracion/Labels_XC237810_1.txt"  # Replace with your manual label file path
output_with_voice = "output_with_voice_huggingface.wav"
output_with_silence = "output_with_silence_huggingface.wav"
vad_output_file = "Labels_XC237810_1_detection.txt"  # The file to save the VAD intervals

# Check if the input file exists
if not os.path.exists(mp3_path):
    print(f"Error: Audio file not found at {mp3_path}")
    exit()

# Detect speech intervals
speech_intervals, duration = detect_speech_intervals(mp3_path, pipeline)
print(f"VAD Intervals (Pipeline): {speech_intervals}")

# Load manual labels
manual_intervals = load_manual_labels(manual_label_file)
if manual_intervals is None:
    exit()
print(f"Manual Intervals: {manual_intervals}")

# Save the VAD intervals to file, including performance metrics
save_vad_intervals(speech_intervals, manual_intervals, vad_output_file)

# Load audio
audio_segment = load_audio(mp3_path)

# Save new audio files with intervals
save_audio_with_intervals(audio_segment, speech_intervals, output_with_voice, output_with_silence, duration)
print("Finished!")